In [58]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

In [4]:
df=pd.read_csv('heart_disease_predic.csv')
print(df.head())
# df=df.drop(columns=['Outcome Variable'])
df.head()

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   0     3       0  
2   0     3       0  
3   1     3       0  
4   3     2       0  


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [15]:
df.isnull().sum()
df.columns = [
    "Age","Sex","ChestPainType","RestingBP","Cholesterol","FastingBS","RestingECG",
    "MaxHR","ExerciseAngina","Oldpeak","ST_Slope","NumMajorVessels","Thalassemia","HeartDisease"
]



print(df.info())
print(df.shape)
df.isnull().sum()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              1025 non-null   int64  
 1   Sex              1025 non-null   int64  
 2   ChestPainType    1025 non-null   int64  
 3   RestingBP        1025 non-null   int64  
 4   Cholesterol      1025 non-null   int64  
 5   FastingBS        1025 non-null   int64  
 6   RestingECG       1025 non-null   int64  
 7   MaxHR            1025 non-null   int64  
 8   ExerciseAngina   1025 non-null   int64  
 9   Oldpeak          1025 non-null   float64
 10  ST_Slope         1025 non-null   int64  
 11  NumMajorVessels  1025 non-null   int64  
 12  Thalassemia      1025 non-null   int64  
 13  HeartDisease     1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB
None
(1025, 14)


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,NumMajorVessels,Thalassemia,HeartDisease
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [30]:
x=df[[
    "Age","Sex","ChestPainType","RestingBP","Cholesterol","FastingBS","RestingECG",
    "MaxHR","ExerciseAngina","Oldpeak","ST_Slope","NumMajorVessels","Thalassemia"]]

y=df['HeartDisease']


preprocess=ColumnTransformer(
    transformers=[
        ('num',StandardScaler(),["Age","RestingBP","Cholesterol","RestingECG","MaxHR","Oldpeak","ST_Slope","NumMajorVessels"]),
        ('cat',OneHotEncoder(),['Sex','ChestPainType','FastingBS','Thalassemia','ExerciseAngina'])
    ]
)

pipe=Pipeline([
    ('step',preprocess),
    ('pca',PCA(n_components=0.95)),
    ('model',DecisionTreeClassifier())
])


x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)


pipe.fit(x_train,y_train)

pred=pipe.predict(x_test)
print('Accuracy Score : ',np.round(accuracy_score(y_test,pred)*100,2),'%')


Accuracy Score :  100.0 %


In [31]:
train_pred = pipe.predict(x_train)
test_pred = pipe.predict(x_test)

print("Train Accuracy:", accuracy_score(y_train, train_pred))
print("Test Accuracy:", accuracy_score(y_test, test_pred))

Train Accuracy: 1.0
Test Accuracy: 1.0


In [36]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipe, x, y, cv=10)
print("Cross Validation Score:", scores.mean() * 100)
print("Scores per fold:", scores * 100)

print(df['Oldpeak'].max())

Cross Validation Score: 99.41176470588235
Scores per fold: [100.         100.         100.         100.         100.
 100.          97.05882353 100.          97.05882353 100.        ]
6.2


In [57]:
age = int(input("Enter Age: "))
sex = input("Enter Sex (Male, Female): ")

if sex.lower()=='male':
    sex=1
else:
    sex=0



cp = int(input("Chest Pain Type (0-3): "))
restbp = int(input("Resting Blood Pressure: "))
chol = int(input("Cholesterol: "))

fbs = input("Fasting Blood Sugar  (Yes,No): ")

if fbs.lower()=='yes':
    fbs=1
else:
    fbs=0

recg = int(input("Resting ECG (0-2): "))
maxhr = int(input("Maximum Heart Rate: "))
exang = int(input("Exercise Angina (1=Yes, 0=No): "))
oldpeak = float(input("Change in heart function after exercise compared to rest (1.0 - 5.0): "))
slope = int(input("ST Slope (0-up)(1-flat)(2-down): "))
vessels = int(input("Number of Major Vessels (0-3): "))
thal = int(input("Thalassemia (1-3): "))


user_input = pd.DataFrame([[age, sex, cp, restbp, chol, fbs, recg, maxhr, exang, oldpeak, slope, vessels, thal]],columns=[
    "Age","Sex","ChestPainType","RestingBP","Cholesterol","FastingBS","RestingECG",
    "MaxHR","ExerciseAngina","Oldpeak","ST_Slope","NumMajorVessels","Thalassemia"])



result=pipe.predict(user_input)[0]
if result==1:
    print('Major Chance of Heart Decies')
else :
    print('Healthy person')

Major Chance of Heart Decies


In [60]:
joblib.dump(pipe,'Heart_Decies_predict.pkl')

['Heart_Decies_predict.pkl']